In [12]:
import albumentations as A
import cv2
import os
from collections import Counter
from sklearn.utils import resample

In [13]:
transform = A.Compose(
    [
        A.Rotate(limit=15, p=1),
        # A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=1),
        # A.GaussianBlur(p=1),
        # A.Perspective(scale=(0.07, 0.1), fit_output=True, pad_mode=cv2.BORDER_CONSTANT, pad_val=1, p=0.5)
    ]
)

In [17]:
data_dir = "../yolov8-dataset-v2-extracted/balanced/test/"
class_names = os.listdir(data_dir)
class_counts = Counter()
for class_name in class_names:
    class_counts[class_name] = len(os.listdir(os.path.join(data_dir, class_name)))
max_count = max(class_counts.values())

In [18]:
print(f'max data count: {max_count}')

max data count: 38


In [19]:
for class_name, count in class_counts.items():
    if count < max_count:
        diff = max_count - count
        image_paths = [
            os.path.join(data_dir, class_name, image_name)
            for image_name in os.listdir(os.path.join(data_dir, class_name))
        ]
        image_paths = resample(
            image_paths, n_samples=diff, random_state=42, replace=True
        )
        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            transformed = transform(image=image)
            transformed_image = transformed["image"]
            new_image_path = os.path.join(data_dir, class_name, f"{i}.jpg")
            transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(new_image_path, transformed_image)